# Week 3 Assignment: Segmenting and Clustering Neighborhoods

## Step 1: Webscraping Wikipedia Webpage

In [1]:
# Install required packages and libraries

from bs4 import BeautifulSoup # this module helps in web scrapping.

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python-3.7-main

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _libgcc_mutex-0.1          |      conda_forge           3 KB  conda-forge
    _openmp_mutex-4.5          |           1_llvm           5 KB  conda-forge
    _py-xgboost-mutex-2.0      |            cpu_0           8 KB  conda-forge
    _pytorch_select-0.2        |            gpu_0           2 KB
    absl-py-0.12.0             |     pyhd8ed1ab_0          96 KB  conda-forge
    aiohttp-3.7.4              |   py37h5e8e339_0         

In [2]:
#this url contains the table listing of postal codes and associated boroughs and neighborhoods in Toronta on Wikipedia that we need to scrape
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [3]:
# get the contents of the webpage in text format and store in a variable called data
data  = requests.get(url).text 

In [4]:
#create soup object
soup = BeautifulSoup(data,"html5lib")

In [5]:
#Firstly create an empty list
table_contents=[]

In [6]:
#Find the table of data
table=soup.find('table')

In [7]:
# Create a dictionary called cell having 3 keys PostalCode, Borough and Neighborhood.
# As postal code contains upto 3 characters extract that using tablerow.p.text
# Next use split,strip and replace functions for getting Borough and Neighborhood information.
# Append to the list.

for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

print(table_contents)

[{'PostalCode': 'M3A', 'Borough': 'North York', 'Neighborhood': 'Parkwoods'}, {'PostalCode': 'M4A', 'Borough': 'North York', 'Neighborhood': 'Victoria Village'}, {'PostalCode': 'M5A', 'Borough': 'Downtown Toronto', 'Neighborhood': 'Regent Park, Harbourfront'}, {'PostalCode': 'M6A', 'Borough': 'North York', 'Neighborhood': 'Lawrence Manor, Lawrence Heights'}, {'PostalCode': 'M7A', 'Borough': "Queen's Park", 'Neighborhood': 'Ontario Provincial Government'}, {'PostalCode': 'M9A', 'Borough': 'Etobicoke', 'Neighborhood': 'Islington Avenue'}, {'PostalCode': 'M1B', 'Borough': 'Scarborough', 'Neighborhood': 'Malvern, Rouge'}, {'PostalCode': 'M3B', 'Borough': 'North York', 'Neighborhood': 'Don Mills North'}, {'PostalCode': 'M4B', 'Borough': 'East York', 'Neighborhood': 'Parkview Hill, Woodbine Gardens'}, {'PostalCode': 'M5B', 'Borough': 'Downtown Toronto', 'Neighborhood': 'Garden District, Ryerson'}, {'PostalCode': 'M6B', 'Borough': 'North York', 'Neighborhood': 'Glencairn'}, {'PostalCode': 'M9

## Step 2: Create Dataframe

In [8]:
# Import pandas
import pandas as pd

In [9]:
# Create a dataframe with list
df=pd.DataFrame(table_contents)
df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                            'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East TorontoBusiness reply mail Processing Cen...,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [10]:
import numpy as np

In [11]:
#If neighborhood is empty or not assigned, then the neighborhood will be the same as the borough.
df['Neighborhood'] = np.where(
   (df['Neighborhood'] == ""), df['Borough'], df['Neighborhood'])

In [12]:
#If neighborhood is empty or not assigned, then the neighborhood will be the same as the borough.
df['Neighborhood'] = np.where(
   (df['Neighborhood'] == "Not assigned"), df['Borough'], df['Neighborhood'])

In [13]:
df.shape

(103, 3)

## Step 3: Get Coordinates

In [14]:
#Using csv file to obtain the coordinates.
coords = pd.read_csv('https://cocl.us/Geospatial_data')

In [15]:
coords

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [16]:
# lets rename Postal Code column in df to be the same as 'coords' dataframe since it will be used as the index for adding the coordinates to df.
df.rename(columns={'PostalCode':'Postal Code'}, inplace=True)
df

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East TorontoBusiness reply mail Processing Cen...,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [17]:
# Adding Latitude and Longitude colums from 'coords' dataframe to df using 'Postal Code' as index
col = ['Postal Code']
df = df.join(coords.set_index(col), on=col)

In [18]:
#Sort table by Postal Code
df = df.sort_values(by=['Postal Code'])

In [19]:
df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
12,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
18,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
22,M1G,Scarborough,Woburn,43.770992,-79.216917
26,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [20]:
# Reset index and use drop parameter to avoid old index from being added as a column
df.reset_index(drop=True)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437


## Step 4: Explore the Venues around Neighborhoods in Toronto

#### Use geopy library to get the latitude and longitude values of Toronto.

In [21]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


#### Create a map of Toronto with neighborhoods superimposed on top.

In [22]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Since there are cases of multiple neighborhoods per borough, lets split multiple neighborhoods into multiple row entries 

In [23]:
df = df.drop('Neighborhood', axis=1).join(df['Neighborhood'].str.split(',', expand=True).stack().reset_index(level=1, drop=True).rename('Neighborhood'))
df

,Postal Code,Borough,Latitude,Longitude,Neighborhood
0,M3A,North York,43.753259,-79.329656,Parkwoods
1,M4A,North York,43.725882,-79.315572,Victoria Village
2,M5A,Downtown Toronto,43.654260,-79.360636,Regent Park
2,M5A,Downtown Toronto,43.654260,-79.360636,Harbourfront
3,M6A,North York,43.718518,-79.464763,Lawrence Manor
...,...,...,...,...,...
102,M8Z,Etobicoke,43.628841,-79.520999,Mimico NW
102,M8Z,Etobicoke,43.628841,-79.520999,The Queensway West
102,M8Z,Etobicoke,43.628841,-79.520999,South of Bloor
102,M8Z,Etobicoke,43.628841,-79.520999,Kingsway Park South West


In [24]:
#Lets drop any duplicates neighborhood entries in the dataframe
df=df.drop_duplicates()
df

,Postal Code,Borough,Latitude,Longitude,Neighborhood
0,M3A,North York,43.753259,-79.329656,Parkwoods
1,M4A,North York,43.725882,-79.315572,Victoria Village
2,M5A,Downtown Toronto,43.654260,-79.360636,Regent Park
2,M5A,Downtown Toronto,43.654260,-79.360636,Harbourfront
3,M6A,North York,43.718518,-79.464763,Lawrence Manor
...,...,...,...,...,...
102,M8Z,Etobicoke,43.628841,-79.520999,Mimico NW
102,M8Z,Etobicoke,43.628841,-79.520999,The Queensway West
102,M8Z,Etobicoke,43.628841,-79.520999,South of Bloor
102,M8Z,Etobicoke,43.628841,-79.520999,Kingsway Park South West


#### Define Foursquare Credentials and Version

In [25]:
CLIENT_ID = 'PTAQCW0BVZPJFP5DXPTZ5QBZBNHIVY2XRNPGGYV5KAUDGF0A' # your Foursquare ID
CLIENT_SECRET = 'CUNGOE0GL0XUU5MCETT1U25TTHECN2ABEO1X5QJR0Y1WV4WY' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: PTAQCW0BVZPJFP5DXPTZ5QBZBNHIVY2XRNPGGYV5KAUDGF0A
CLIENT_SECRET:CUNGOE0GL0XUU5MCETT1U25TTHECN2ABEO1X5QJR0Y1WV4WY


#### Let's create a function to explore all neighborhourds in Toronto

In [26]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called _toronto_venues_.

In [27]:
# type your answer here
toronto_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park
 Harbourfront
Lawrence Manor
 Lawrence Heights
Ontario Provincial Government
Islington Avenue
Malvern
 Rouge
Don Mills North
Parkview Hill
 Woodbine Gardens
Garden District
 Ryerson
Glencairn
West Deane Park
 Princess Gardens
 Martin Grove
 Islington
 Cloverdale
Rouge Hill
 Port Union
 Highland Creek
Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate
 Bloordale Gardens
 Old Burnhamthorpe
 Markland Wood
Guildwood
 Morningside
 West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor
 Wilson Heights
 Downsview North
Thorncliffe Park
Richmond
 Adelaide
 King
Dufferin
 Dovercourt Village
Scarborough Village
Fairview
 Henry Farm
 Oriole
Northwood Park
 York University
The Danforth  East
Harbourfront East
 Union Station
 Toronto Islands
Little Portugal
 Trinity
Kennedy Park
 Ionview
 East Birchmount Park
Bayview Village
Downsview East
The Danforth

#### Let's check the size of the resulting dataframe

In [28]:
print(toronto_venues.shape)
toronto_venues.head()

(4230, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


Let's check how many venues were returned for each neighborhood

In [29]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Adelaide,92,92,92,92,92,92
Agincourt North,4,4,4,4,4,4
Albion Gardens,11,11,11,11,11,11
Bathurst Quay,16,16,16,16,16,16
Beaumond Heights,11,11,11,11,11,11
...,...,...,...,...,...,...
Willowdale West,4,4,4,4,4,4
Woburn,3,3,3,3,3,3
Woodbine Heights,6,6,6,6,6,6


#### Let's find out how many unique categories can be curated from all the returned venues

In [30]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 269 uniques categories.


## Step 5: Analyze Each Neighborhood Venue Category

In [31]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# shift column 'Name' to first position
first_column = toronto_onehot.pop('Neighborhood')
  
# insert column using insert(position,column_name,
# first_column) function
toronto_onehot.insert(0, 'Neighborhood', first_column)

toronto_onehot.head()

,Neighborhood,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Train Station,Truck Stop,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


And let's examine the new dataframe size.

In [32]:
toronto_onehot.shape

(4230, 269)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [33]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Train Station,Truck Stop,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Adelaide,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.01087,0.0,...,0.0,0.0,0.01087,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Agincourt North,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.00000,0.0,...,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Albion Gardens,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.00000,0.0,...,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Bathurst Quay,0.0,0.0625,0.0625,0.0625,0.125,0.125,0.125,0.00000,0.0,...,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Beaumond Heights,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.00000,0.0,...,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207,Willowdale West,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.00000,0.0,...,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
208,Woburn,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.00000,0.0,...,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
209,Woodbine Heights,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.00000,0.0,...,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
210,York Mills,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.00000,0.0,...,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Let's write a function to sort the venues in descending order.

In [34]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [35]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Coffee Shop,Café,Restaurant,Thai Restaurant,Clothing Store,Hotel,Gym,Deli / Bodega,Bakery,Salad Place
1,Agincourt North,Intersection,Playground,Park,Asian Restaurant,Mobile Phone Shop,Motel,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant
2,Albion Gardens,Pizza Place,Grocery Store,Beer Store,Fried Chicken Joint,Sandwich Place,Coffee Shop,Discount Store,Fast Food Restaurant,Pharmacy,Motel
3,Bathurst Quay,Airport Lounge,Airport Service,Airport Terminal,Harbor / Marina,Airport,Bar,Coffee Shop,Boat or Ferry,Rental Car Location,Sculpture Garden
4,Beaumond Heights,Pizza Place,Grocery Store,Beer Store,Fried Chicken Joint,Sandwich Place,Coffee Shop,Discount Store,Fast Food Restaurant,Pharmacy,Motel


## Step 6: Cluster Neighborhoods

Run _k_-means to cluster the neighborhood into 5 clusters.

In [36]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 
toronto_grouped_clustering

,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Train Station,Truck Stop,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.01087,0.0,0.0,...,0.0,0.0,0.01087,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.00000,0.0,0.0,...,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.00000,0.0,0.0,...,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0625,0.0625,0.0625,0.125,0.125,0.125,0.00000,0.0,0.0,...,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.00000,0.0,0.0,...,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.00000,0.0,0.0,...,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
208,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.00000,0.0,0.0,...,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
209,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.00000,0.0,0.0,...,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
210,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.00000,0.0,0.0,...,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [37]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
toronto_merged.head() # check the last columns!

,Postal Code,Borough,Latitude,Longitude,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,43.753259,-79.329656,Parkwoods,4.0,Food & Drink Shop,Park,Middle Eastern Restaurant,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop,Accessories Store
1,M4A,North York,43.725882,-79.315572,Victoria Village,1.0,Pizza Place,Portuguese Restaurant,Hockey Arena,Intersection,Coffee Shop,Accessories Store,Modern European Restaurant,Movie Theater,Motel,Moroccan Restaurant
2,M5A,Downtown Toronto,43.654260,-79.360636,Regent Park,1.0,Coffee Shop,Park,Bakery,Pub,Breakfast Spot,Theater,Café,Yoga Studio,Greek Restaurant,Event Space
2,M5A,Downtown Toronto,43.654260,-79.360636,Harbourfront,1.0,Coffee Shop,Park,Bakery,Pub,Breakfast Spot,Theater,Café,Yoga Studio,Greek Restaurant,Event Space
3,M6A,North York,43.718518,-79.464763,Lawrence Manor,1.0,Clothing Store,Furniture / Home Store,Accessories Store,Vietnamese Restaurant,Boutique,Coffee Shop,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop


In [38]:
toronto_merged.reset_index(drop=True)

,Postal Code,Borough,Latitude,Longitude,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,43.753259,-79.329656,Parkwoods,4.0,Food & Drink Shop,Park,Middle Eastern Restaurant,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop,Accessories Store
1,M4A,North York,43.725882,-79.315572,Victoria Village,1.0,Pizza Place,Portuguese Restaurant,Hockey Arena,Intersection,Coffee Shop,Accessories Store,Modern European Restaurant,Movie Theater,Motel,Moroccan Restaurant
2,M5A,Downtown Toronto,43.654260,-79.360636,Regent Park,1.0,Coffee Shop,Park,Bakery,Pub,Breakfast Spot,Theater,Café,Yoga Studio,Greek Restaurant,Event Space
3,M5A,Downtown Toronto,43.654260,-79.360636,Harbourfront,1.0,Coffee Shop,Park,Bakery,Pub,Breakfast Spot,Theater,Café,Yoga Studio,Greek Restaurant,Event Space
4,M6A,North York,43.718518,-79.464763,Lawrence Manor,1.0,Clothing Store,Furniture / Home Store,Accessories Store,Vietnamese Restaurant,Boutique,Coffee Shop,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211,M8Z,Etobicoke,43.628841,-79.520999,Mimico NW,1.0,Convenience Store,Thrift / Vintage Store,Burrito Place,Social Club,Fast Food Restaurant,Sandwich Place,Supplement Shop,Kids Store,Bakery,Discount Store
212,M8Z,Etobicoke,43.628841,-79.520999,The Queensway West,1.0,Convenience Store,Thrift / Vintage Store,Burrito Place,Social Club,Fast Food Restaurant,Sandwich Place,Supplement Shop,Kids Store,Bakery,Discount Store
213,M8Z,Etobicoke,43.628841,-79.520999,South of Bloor,1.0,Convenience Store,Thrift / Vintage Store,Burrito Place,Social Club,Fast Food Restaurant,Sandwich Place,Supplement Shop,Kids Store,Bakery,Discount Store
214,M8Z,Etobicoke,43.628841,-79.520999,Kingsway Park South West,1.0,Convenience Store,Thrift / Vintage Store,Burrito Place,Social Club,Fast Food Restaurant,Sandwich Place,Supplement Shop,Kids Store,Bakery,Discount Store


Lets check clusterlabels 

In [55]:
toronto_merged['Cluster Labels'].isna().sum()

0

In [56]:
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].apply(np.int64)
toronto_merged.reset_index(drop=True)
toronto_merged


,Postal Code,Borough,Latitude,Longitude,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,43.753259,-79.329656,Parkwoods,4,Food & Drink Shop,Park,Middle Eastern Restaurant,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop,Accessories Store
1,M4A,North York,43.725882,-79.315572,Victoria Village,1,Pizza Place,Portuguese Restaurant,Hockey Arena,Intersection,Coffee Shop,Accessories Store,Modern European Restaurant,Movie Theater,Motel,Moroccan Restaurant
2,M5A,Downtown Toronto,43.654260,-79.360636,Regent Park,1,Coffee Shop,Park,Bakery,Pub,Breakfast Spot,Theater,Café,Yoga Studio,Greek Restaurant,Event Space
2,M5A,Downtown Toronto,43.654260,-79.360636,Harbourfront,1,Coffee Shop,Park,Bakery,Pub,Breakfast Spot,Theater,Café,Yoga Studio,Greek Restaurant,Event Space
3,M6A,North York,43.718518,-79.464763,Lawrence Manor,1,Clothing Store,Furniture / Home Store,Accessories Store,Vietnamese Restaurant,Boutique,Coffee Shop,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102,M8Z,Etobicoke,43.628841,-79.520999,Mimico NW,1,Convenience Store,Thrift / Vintage Store,Burrito Place,Social Club,Fast Food Restaurant,Sandwich Place,Supplement Shop,Kids Store,Bakery,Discount Store
102,M8Z,Etobicoke,43.628841,-79.520999,The Queensway West,1,Convenience Store,Thrift / Vintage Store,Burrito Place,Social Club,Fast Food Restaurant,Sandwich Place,Supplement Shop,Kids Store,Bakery,Discount Store
102,M8Z,Etobicoke,43.628841,-79.520999,South of Bloor,1,Convenience Store,Thrift / Vintage Store,Burrito Place,Social Club,Fast Food Restaurant,Sandwich Place,Supplement Shop,Kids Store,Bakery,Discount Store
102,M8Z,Etobicoke,43.628841,-79.520999,Kingsway Park South West,1,Convenience Store,Thrift / Vintage Store,Burrito Place,Social Club,Fast Food Restaurant,Sandwich Place,Supplement Shop,Kids Store,Bakery,Discount Store


Finally, let's visualize the resulting clusters

In [57]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Step 7: Examine Clusters

Lets examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

#### Cluster 1

In [58]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Scarborough,0,Construction & Landscaping,Bar,Accessories Store,Mobile Phone Shop,Motel,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Miscellaneous Shop
12,Scarborough,0,Construction & Landscaping,Bar,Accessories Store,Mobile Phone Shop,Motel,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Miscellaneous Shop
12,Scarborough,0,Construction & Landscaping,Bar,Accessories Store,Mobile Phone Shop,Motel,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Miscellaneous Shop
87,Downtown Toronto,0,Airport Lounge,Airport Service,Airport Terminal,Harbor / Marina,Airport,Bar,Coffee Shop,Boat or Ferry,Rental Car Location,Sculpture Garden
87,Downtown Toronto,0,Airport Lounge,Airport Service,Airport Terminal,Harbor / Marina,Airport,Bar,Coffee Shop,Boat or Ferry,Rental Car Location,Sculpture Garden
87,Downtown Toronto,0,Airport Lounge,Airport Service,Airport Terminal,Harbor / Marina,Airport,Bar,Coffee Shop,Boat or Ferry,Rental Car Location,Sculpture Garden
87,Downtown Toronto,0,Airport Lounge,Airport Service,Airport Terminal,Harbor / Marina,Airport,Bar,Coffee Shop,Boat or Ferry,Rental Car Location,Sculpture Garden
87,Downtown Toronto,0,Airport Lounge,Airport Service,Airport Terminal,Harbor / Marina,Airport,Bar,Coffee Shop,Boat or Ferry,Rental Car Location,Sculpture Garden
87,Downtown Toronto,0,Airport Lounge,Airport Service,Airport Terminal,Harbor / Marina,Airport,Bar,Coffee Shop,Boat or Ferry,Rental Car Location,Sculpture Garden
87,Downtown Toronto,0,Airport Lounge,Airport Service,Airport Terminal,Harbor / Marina,Airport,Bar,Coffee Shop,Boat or Ferry,Rental Car Location,Sculpture Garden


#### Cluster 2

In [59]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,1,Pizza Place,Portuguese Restaurant,Hockey Arena,Intersection,Coffee Shop,Accessories Store,Modern European Restaurant,Movie Theater,Motel,Moroccan Restaurant
2,Downtown Toronto,1,Coffee Shop,Park,Bakery,Pub,Breakfast Spot,Theater,Café,Yoga Studio,Greek Restaurant,Event Space
2,Downtown Toronto,1,Coffee Shop,Park,Bakery,Pub,Breakfast Spot,Theater,Café,Yoga Studio,Greek Restaurant,Event Space
3,North York,1,Clothing Store,Furniture / Home Store,Accessories Store,Vietnamese Restaurant,Boutique,Coffee Shop,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop
3,North York,1,Clothing Store,Furniture / Home Store,Accessories Store,Vietnamese Restaurant,Boutique,Coffee Shop,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop
...,...,...,...,...,...,...,...,...,...,...,...,...
102,Etobicoke,1,Convenience Store,Thrift / Vintage Store,Burrito Place,Social Club,Fast Food Restaurant,Sandwich Place,Supplement Shop,Kids Store,Bakery,Discount Store
102,Etobicoke,1,Convenience Store,Thrift / Vintage Store,Burrito Place,Social Club,Fast Food Restaurant,Sandwich Place,Supplement Shop,Kids Store,Bakery,Discount Store
102,Etobicoke,1,Convenience Store,Thrift / Vintage Store,Burrito Place,Social Club,Fast Food Restaurant,Sandwich Place,Supplement Shop,Kids Store,Bakery,Discount Store
102,Etobicoke,1,Convenience Store,Thrift / Vintage Store,Burrito Place,Social Club,Fast Food Restaurant,Sandwich Place,Supplement Shop,Kids Store,Bakery,Discount Store


#### Cluster 3

In [60]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
57,North York,2,Baseball Field,Accessories Store,Mobile Phone Shop,Motel,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Miscellaneous Shop,Martial Arts School
57,North York,2,Baseball Field,Accessories Store,Mobile Phone Shop,Motel,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Miscellaneous Shop,Martial Arts School
101,Etobicoke,2,Baseball Field,Accessories Store,Mobile Phone Shop,Motel,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Miscellaneous Shop,Martial Arts School
101,Etobicoke,2,Baseball Field,Accessories Store,Mobile Phone Shop,Motel,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Miscellaneous Shop,Martial Arts School
101,Etobicoke,2,Baseball Field,Accessories Store,Mobile Phone Shop,Motel,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Miscellaneous Shop,Martial Arts School
101,Etobicoke,2,Baseball Field,Accessories Store,Mobile Phone Shop,Motel,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Miscellaneous Shop,Martial Arts School
101,Etobicoke,2,Baseball Field,Accessories Store,Mobile Phone Shop,Motel,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Miscellaneous Shop,Martial Arts School
101,Etobicoke,2,Baseball Field,Accessories Store,Mobile Phone Shop,Motel,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Miscellaneous Shop,Martial Arts School
101,Etobicoke,2,Baseball Field,Accessories Store,Mobile Phone Shop,Motel,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Miscellaneous Shop,Martial Arts School
101,Etobicoke,2,Baseball Field,Accessories Store,Mobile Phone Shop,Motel,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Miscellaneous Shop,Martial Arts School


#### Cluster 4

In [61]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Etobicoke,3,Bakery,Accessories Store,Mobile Phone Shop,Motel,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Miscellaneous Shop,Martial Arts School
11,Etobicoke,3,Bakery,Accessories Store,Mobile Phone Shop,Motel,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Miscellaneous Shop,Martial Arts School
11,Etobicoke,3,Bakery,Accessories Store,Mobile Phone Shop,Motel,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Miscellaneous Shop,Martial Arts School
11,Etobicoke,3,Bakery,Accessories Store,Mobile Phone Shop,Motel,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Miscellaneous Shop,Martial Arts School
11,Etobicoke,3,Bakery,Accessories Store,Mobile Phone Shop,Motel,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Miscellaneous Shop,Martial Arts School


#### Cluster 5

In [62]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,4,Food & Drink Shop,Park,Middle Eastern Restaurant,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop,Accessories Store
21,York,4,Park,Women's Store,Pool,Accessories Store,Miscellaneous Shop,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop
45,North York,4,Park,Accessories Store,Miscellaneous Shop,Motel,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Middle Eastern Restaurant
45,North York,4,Park,Accessories Store,Miscellaneous Shop,Motel,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Middle Eastern Restaurant
52,North York,4,Park,Accessories Store,Miscellaneous Shop,Motel,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Middle Eastern Restaurant
52,North York,4,Park,Accessories Store,Miscellaneous Shop,Motel,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Middle Eastern Restaurant
64,York,4,Park,Accessories Store,Miscellaneous Shop,Motel,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Middle Eastern Restaurant
66,North York,4,Park,Convenience Store,Accessories Store,Mobile Phone Shop,Motel,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Middle Eastern Restaurant
91,Downtown Toronto,4,Park,Playground,Trail,Miscellaneous Shop,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Middle Eastern Restaurant
98,Etobicoke,4,Park,River,Accessories Store,Miscellaneous Shop,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Middle Eastern Restaurant
